# Imports

## Python Libraries

In [1]:
import numpy as np
import os
import pandas as pd
import pickle
import torch
import torch.nn.functional as F

from copy import deepcopy
from datetime import datetime
from PIL import Image
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader,Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms



/home/ibs/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2024-06-10 19:44:56.226379: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 19:44:57.150234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Importing the datasets

In [2]:
dataset = pd.read_csv('Datasets/Products.csv', lineterminator='\n')

# Cleaning the Dataset

## Cleaning the table

In [3]:
dataset.count(), dataset.dropna().count(), dataset["price"].map(lambda x: x[0]).unique(), dataset["price"].map(lambda x: x[-3:]).unique()

(Unnamed: 0             7156
 id                     7156
 product_name           7156
 category               7156
 product_description    7156
 price                  7156
 location               7156
 dtype: int64,
 Unnamed: 0             7156
 id                     7156
 product_name           7156
 category               7156
 product_description    7156
 price                  7156
 location               7156
 dtype: int64,
 array(['£'], dtype=object),
 array(['.00', '.99', '.78', '.01', '.97', '.25', '.50', '.20', '.90',
        '.80', '.60', '.23', '.05', '.75', '.56', '.40', '.44', '.95',
        '.66', '.35', '.85', '.30', '.45', '.16', '.69', '.49', '.55',
        '.09', '.11'], dtype=object))

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7156 entries, 0 to 7155
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           7156 non-null   int64 
 1   id                   7156 non-null   object
 2   product_name         7156 non-null   object
 3   category             7156 non-null   object
 4   product_description  7156 non-null   object
 5   price                7156 non-null   object
 6   location             7156 non-null   object
dtypes: int64(1), object(6)
memory usage: 391.5+ KB


# So the dataset seems to already be clean

Now we convert the price column

In [5]:
dataset_cleaned = dataset

def remove_pound_sign(string_to_replace) -> str:
    return 

dataset_cleaned['price'] = dataset_cleaned['price'].map(lambda x : x.replace('£', '')) #Removes the pound signs
dataset_cleaned['price'] = dataset_cleaned['price'].map(lambda x: x.replace(',', '')) #Removes commas

dataset_cleaned['price'] = pd.to_numeric(dataset_cleaned['price'])

# Now we extract the root category from each item

In [6]:
dataset_cleaned['category'] = dataset_cleaned['category'].map(lambda x: x.split(' /' )[0])

In [7]:
dataset_cleaned['category'].unique()

array(['Home & Garden', 'Baby & Kids Stuff', 'DIY Tools & Materials',
       'Music, Films, Books & Games', 'Phones, Mobile Phones & Telecoms',
       'Clothes, Footwear & Accessories', 'Other Goods',
       'Health & Beauty', 'Sports, Leisure & Travel', 'Appliances',
       'Computers & Software', 'Office Furniture & Equipment',
       'Video Games & Consoles'], dtype=object)

Now we create an encoder to go to and from the categories and integers

In [8]:
list_of_categories = list(dataset_cleaned['category'].unique())

encoder = {x: list_of_categories.index(x) for x in list_of_categories}

decoder = {list_of_categories.index(x):x for x in list_of_categories}

encoder, decoder

#Let's save these to pickle files:

with open("encoder_pickle", 'wb') as encody:
    pickle.dump(encoder, encody)

with open("decoder_pickle", 'wb') as decody:
    pickle.dump(decoder, decody)

# Now we merge this with the original table:

In [9]:
dataset_cleaned['root_category'] = dataset_cleaned['category']
dataset_cleaned['root_category_index'] = dataset_cleaned['category'].map(lambda x:encoder[x])

# Next we merge this with the images table:

In [10]:
images_dataset = pd.read_csv('Datasets/Images.csv')

dataset_cleaned["merge_column"] = dataset_cleaned['id']
images_dataset['merge_column'] = images_dataset['product_id']

dataset_cleaned.dtypes, images_dataset.dtypes

(Unnamed: 0               int64
 id                      object
 product_name            object
 category                object
 product_description     object
 price                  float64
 location                object
 root_category           object
 root_category_index      int64
 merge_column            object
 dtype: object,
 Unnamed: 0       int64
 id              object
 product_id      object
 merge_column    object
 dtype: object)

In [11]:
merged_df = images_dataset.merge(dataset_cleaned, on='merge_column')

In [12]:
merged_df = merged_df[["id_x", "product_id", "root_category", "root_category_index"]]
merged_df = merged_df.rename(columns={'id_x':'id'})
merged_df = merged_df.drop(columns=['product_id'])

merged_df.dtypes

id                     object
root_category          object
root_category_index     int64
dtype: object

In [13]:
merged_df.to_csv('Datasets/training_data.csv', index=False)

## Clean Image Dataset:

In [14]:
def resize_image(final_size, im):
    size = im.size
    ratio = float(final_size) / max(size)
    new_image_size = tuple([int(x*ratio) for x in size])
    im = im.resize(new_image_size)
    new_im = Image.new("RGB", (final_size, final_size))
    new_im.paste(im, ((final_size-new_image_size[0])//2, (final_size-new_image_size[1])//2))
    return new_im

def clean_images(path_to_extract = "Datasets/images/", path_to_save = "Datasets/cleaned_images/", image_size = 64):
    dirs = os.listdir(path_to_extract)
    final_size = image_size
    for n, item in enumerate(dirs, 1):
        #print(n, item)
        im = Image.open(path_to_extract + item)
        #print(im.width, im.height)
        new_im = resize_image(final_size, im)
        #print(new_im.width, new_im.height)
        new_im.save(path_to_save + item)


In [15]:
#clean_images(path_to_save="Datasets/cleaned_images_64/")

# Working with the Model

## Casting into a PyTorch dataset

In [18]:
class ImageDataset(Dataset):
    def __init__(self, df_of_keys:pd.DataFrame, folder_of_images:str) -> None:
        super().__init__()

        # Set the labels to be the column 'root_cotegory_index' of df_of_keys
        self.labels = df_of_keys['root_category_index']

        # Assings image_paths to the file name from the column 'id' of df_of_keys and maps it to it's path relative to the project root folder
        self.image_paths = df_of_keys['id'].map(lambda x : folder_of_images + x + '.jpg')

        # This will be used later to turn images from self.image_paths into tensors
        self.image_transformer = transforms.PILToTensor()
    
    def __getitem__(self, index):
        
        # Opens the image at index with using PIL.Image
        with Image.open(self.image_paths[index]) as img:

            # Sets the feature to be a tensor obtained by applying PILToTensor to the relevant image
            X = self.image_transformer(img)
        
        # y is the label from self.labels
        y = self.labels[index]

        #print(X.shape, y.shape)
        return X, y
    
    def __len__(self):
        return len(self.labels)



In [19]:
my_dataset = ImageDataset(merged_df, "Datasets/cleaned_images_64/")

# A quick sanity check of the dataset:
my_dataset[1][0], my_dataset[1][1].shape, len(my_dataset)

(tensor([[[0, 0, 0,  ..., 0, 0, 1],
          [0, 0, 0,  ..., 0, 1, 1],
          [0, 0, 0,  ..., 0, 1, 3],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[2, 1, 0,  ..., 1, 1, 0],
          [2, 1, 0,  ..., 1, 0, 0],
          [2, 1, 0,  ..., 1, 0, 0],
          ...,
          [2, 2, 1,  ..., 1, 0, 0],
          [2, 2, 1,  ..., 1, 0, 0],
          [2, 2, 1,  ..., 1, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 2, 2],
          [0, 0, 0,  ..., 0, 2, 2],
          [0, 0, 0,  ..., 0, 2, 2]]], dtype=torch.uint8),
 (),
 12604)

## Splitting the dataset:

In [20]:
train_dataset, test_dataset = torch.utils.data.random_split(
    my_dataset, [10000, len(my_dataset) - 10000],
)

train_dataset, validation_dataset = torch.utils.data.random_split(
    train_dataset, [8000, 2000]
)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=16)
validation_loader = DataLoader(validation_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

/home/ibs/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Defining the model

In [21]:
# Sets the device for PyTorch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Loads the resnet50 model
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', model = 'nvidia_resnet50', pretrained = True )
# Don't know what this does
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')
# Again don't know what this does
resnet50.eval().to(device)

torch.cuda.is_available()

Using cache found in /home/ibs/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/ibs/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/ibs/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
Using cache found in /home/ibs/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


True

## Examining the model

Let's print out the layers of resnet50:

In [22]:
resnet50.modules

<bound method Module.modules of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layers): Sequential(
    (0): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): S

In [23]:
for item in resnet50.modules():
    print(item)
    pass

resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layers): Sequential(
    (0): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layers): Sequential(
    (0): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

- The final layer, `fc` takes in 2048 features and outputs 1000 features. The number of labels we need to classify, i.e. the number of distinct values in `merged_df['root_categories']` is 13.

- So we need to modify the `fc` layer so that it has 13 outputs. We do this by changing `fc` to `torch.nn.Linear(2048, 13)`.

In [24]:
class ImageClassifier(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        self.resnet50.fc = torch.nn.Linear(2048, 13)
        
        # Freeze all the layers, by setting 'requires_grad = False'
        for param in self.resnet50.parameters():
            param.requires_grad = False
        
        # Unfreeze the final layer, by setting 'requires_grad = False' for the fc layer
        for param in self.resnet50.fc.parameters():
            param.requires_grad = True
    
    def forward(self, X):
        return F.softmax(self.resnet50(X.float()))

In [25]:
model = ImageClassifier()

Using cache found in /home/ibs/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [63]:
# The training loop. Takes in a model, the training and validation data loaders, the number of epochs and the initial learning rate
def train(model, train_loader, validation_loader, epochs = 10, learning_rate = 1):

    # Set the optimiser to be an instance of the stochastic gradient descent class
    optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate)

    # Define a learning rate scheduler as an instance of the ReduceLROnPlateau class
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode = 'min', patience = 50, cooldown = 20, min_lr = 0.0001)

    # Writer will be used to track model performance with TensorBoard
    writer = SummaryWriter()

    # Keep track of the number of batches to plot model performace against
    batch_index = 0
    
    # Prints an validation score
    print(f"Initial validation accuracy score{accuracy_score_from_valiadation(model, validation_loader)}")

    # Loop over the number of epochs
    for epoch in range(epochs):

        # Within each epoch, we pass through the entire training data in batches indexed by batch
        for batch in train_loader:
            features, labels = batch

            # Calculate the loss via cross_entropy
            loss = F.cross_entropy(model(features), labels)

            # Create the grad attributes
            loss.backward() 

            # Print the performance
            print(f"Epoch: {epoch}, batch index: {batch_index}, learning rate: {scheduler.get_last_lr()}, loss:{loss.item()}")

            # Perform one step of stochastic gradient descent
            optimiser.step()

            # Zero the gradients (Apparently set_to_none=True imporves performace)
            optimiser.zero_grad(set_to_none=True)

            # Feed the loss amount into the learning rate scheduler to decide the next learning rate
            scheduler.step(loss.item())

            # Write the performance to the TensorBoard plot
            writer.add_scalar('loss', loss.item(), batch_index)

            # Increment the batch index
            batch_index += 1
        
        # Print the validation loss
        accuracy = accuracy_score_from_valiadation(model, validation_loader)
        print(f"Epoch {epoch}, validation accuracy score{accuracy}")

        # Create an instance of the datetime class
        dt = datetime.now()
        date_stamp = str(dt).replace(':', '_').replace('.', '_').replace(' ', '_')

        # Save the model parameters to the folder 'model_evaluation/weights', along with the time and epoch they were generated
        torch.save(model.state_dict(), f'model_evaluation/weights/model_{date_stamp}_epoch_{epoch}_accuracy_{accuracy}.pt') # Is there a better way to do this?

def accuracy_score_from_valiadation(model, validation_loader):
    validation_data = next(iter(validation_loader))
    predictions = [torch.argmax(prediction) for prediction in model(next(iter(validation_loader))[0])]
    return accuracy_score(predictions, validation_data[1])


In [ ]:
# Train the model
train(model, train_loader, validation_loader, epochs=10, learning_rate=0.001)

# Save the model parameters
torch.save(model.parameters(), 'model_evaluation/final_model/image_model.pt')

# Feature extraction model

In [26]:
feature_extraction_model = deepcopy(model)

feature_extraction_model.resnet50.fc = torch.nn.Linear(2048, 1000)

feature_extraction_model(my_dataset[1][0].unsqueeze(0))

# Testing the image processing script

In [32]:
from image_processor import process_image

image_tensor = process_image("Datasets/images/ffff23f1-59fc-47bd-b0cd-186933803287.jpg")

model(image_tensor)

/tmp/ipykernel_23281/1248834949.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.resnet50(X.float()))


tensor([[0.0672, 0.0817, 0.0877, 0.0646, 0.0701, 0.0923, 0.0742, 0.0578, 0.0786,
         0.0743, 0.0816, 0.0900, 0.0799]], grad_fn=<SoftmaxBackward0>)